# Run EvoCDA Millipede Model

### Prepare Data Input

You will only need the encoding output from the previous step. You can also download the pre-computed encoding from [Zenodo](https://doi.org/10.5281/zenodo.13737880) at path: 
- CRISPR-CLEAR-data/data/encoding_output/20240816_v0_1_84_EvoCDA_encoding_dataframes_denoised_removed_ABE8e_encodings_rep0.tsv
- CRISPR-CLEAR-data/data/encoding_output/20240816_v0_1_84_EvoCDA_encoding_dataframes_denoised_removed_ABE8e_encodings_rep1.tsv
- CRISPR-CLEAR-data/data/encoding_output/20240816_v0_1_84_EvoCDA_encoding_dataframes_denoised_removed_ABE8e_encodings_rep3.tsv

### Import packages

In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "1"


from crispr_millipede import encoding as cme
from crispr_millipede import modelling as cmm

import pandas as pd
import numpy as np

from Bio import SeqIO
from Bio.Seq import Seq

import matplotlib.pyplot as plt
import logomaker
import seaborn as sns
import matplotlib.patches as patches

In [2]:
# Set amplicon sequence - UPDATE
amplicon = "ACATGCTCTAGTGAAAGCCAGTCTGGGCAGCTGGGTAGCTAATGAGGGGATTAGAGAGATTTTGTTGAATGAAAGGCAGATTGAGTCCTGCTACTCGCCCCCTTCATTCCCCTTCATTCATGCCTCATTCTTCCGCCTCCCAGCCGCCTCAACTGGCCAAAGGGAAGTGGAGGCCCTGCCACCTGTAGGGAGGGTCCCCTGGGGCTTGCCCACAGCAAACAGGAAGTCACAGCCTGGTGAGATGGGCCTGGGAATCAGCCACTGAGAAAGTGGGTCTCTTGGGTCCCTGAATTCTTTTTCTGAGTCCCTGCAGCAGTGAAAAAGACACAGAGGCACATAGAGAGTG"
print(f"Length of amplicon {len(amplicon)}")

Length of amplicon 346


### Prepare Millipede specification parameters

In [3]:
design_matrix_spec = cmm.MillipedeDesignMatrixProcessingSpecification(
    wt_normalization=False,
    total_normalization=True,
    sigma_scale_normalized=True,
    decay_sigma_scale=True,
    K_enriched=5,
    K_baseline=5,
    a_parameter=0.0005,
    set_offset_as_presort = True,
    offset_normalized = False,
    offset_psuedocount = 1
)

millipede_model_specification_set = {
    "joint_replicate_per_experiment_models" : cmm.MillipedeModelSpecification(
        model_types=[cmm.MillipedeModelType.NORMAL_SIGMA_SCALED, cmm.MillipedeModelType.NORMAL],
        replicate_merge_strategy=cmm.MillipedeReplicateMergeStrategy.COVARIATE,
        experiment_merge_strategy=cmm.MillipedeExperimentMergeStrategy.SEPARATE,
        S = 5,
        tau = 0.01,
        tau_intercept = 0.0001,
        cutoff_specification=cmm.MillipedeCutoffSpecification(
            per_replicate_each_condition_num_cutoff = 0, 
            per_replicate_all_condition_num_cutoff = 1, 
            all_replicate_num_cutoff = 0, 
            all_experiment_num_cutoff = 0,
            baseline_pop_all_condition_each_replicate_num_cutoff = 3,
            baseline_pop_all_condition_acceptable_rep_count = 2,
            enriched_pop_all_condition_each_replicate_num_cutoff = 3,
            enriched_pop_all_condition_acceptable_rep_count = 2,
            presort_pop_all_condition_each_replicate_num_cutoff = 3,
            presort_pop_all_condition_acceptable_rep_count = 2,
            
        ),
        design_matrix_processing_specification=design_matrix_spec
    )
}

In [4]:
# Load in the encoding data - UPDATE WITH YOUR OWN FILEPATHS and UPDATE SUFFIXES with what was specified in the encoding step
paired_end_experiments_inputdata_denoised = cmm.MillipedeInputDataExperimentalGroup(
    data_directory="./", 
    enriched_pop_fn_experiment_list = ["20240816_v0_1_84_EvoCDA_encoding_dataframes_denoised_removed_ABE8e_encodings_rep{}.tsv"],
    enriched_pop_df_reads_colname = "#Reads_CD19minus",
    baseline_pop_fn_experiment_list = ["20240816_v0_1_84_EvoCDA_encoding_dataframes_denoised_removed_ABE8e_encodings_rep{}.tsv"],
    baseline_pop_df_reads_colname = "#Reads_CD19plus", 
    presort_pop_fn_experiment_list = ["20240816_v0_1_84_EvoCDA_encoding_dataframes_denoised_removed_ABE8e_encodings_rep{}.tsv"],
    presort_pop_df_reads_colname = '#Reads_presort',
    experiment_labels = ["EvoCDA"],
    reps = [0,1,2],
    millipede_model_specification_set = millipede_model_specification_set
   )


Performing initial input validation checks...
Passed validation.
Retrieving data for
	Replicate Merge Strategy: MillipedeReplicateMergeStrategy.COVARIATE 
	Experiment Merge Strategy MillipedeExperimentMergeStrategy.SEPARATE
	Cutoff: 
                    per_replicate_each_condition_num_cutoff=0, 
                    per_replicate_presort_condition_num_cutoff=0, 
                    per_replicate_all_condition_num_cutoff=1, 
                    all_replicate_num_cutoff=0, 
                    all_experiment_num_cutoff=0, 
                    baseline_pop_per_condition_acceptable_rep_count=0, 
                    baseline_pop_per_condition_acceptable_rep_count=0, 
                    baseline_pop_per_condition_acceptable_rep_count=0,
                    enriched_pop_per_condition_each_replicate_num_cutoff=0,
                    enriched_pop_per_condition_acceptable_rep_count=0,
                    presort_pop_per_condition_each_replicate_num_cutoff=0,
                    presort_pop_per_

## Run Millipede

In [28]:
%%time
from crispr_millipede import encoding as cme
from crispr_millipede import modelling as cmm

# Run Millipede on the GPU, could also run on CPU by setting device=cmm.MillipedeComputeDevice.CPU 
paired_end_experiments_models_denoised = cmm.MillipedeModelExperimentalGroup(experiments_inputdata=paired_end_experiments_inputdata_denoised, device=cmm.MillipedeComputeDevice.GPU)

Start model inference for all provided model specifications: 1 total
Starting model inference for model specification id 1/1: joint_replicate_per_experiment_models
Number of single matrices: 1
With 2 model types, the total models to inference for this model specification: 2
Running model(s) for single matrix index: 1/1
Iterating through all 2 provided models: 
Preparing data for model NORMAL_SIGMA_SCALED, 1/2
Running model NORMAL_SIGMA_SCALED


  0%|          | 0/5500 [00:00<?, ?it/s]

Preparing data for model NORMAL, 2/2
Running model NORMAL


  0%|          | 0/5500 [00:00<?, ?it/s]

CPU times: user 39.9 s, sys: 862 ms, total: 40.7 s
Wall time: 38.8 s


In [25]:
# Get the Beta and PIP coefficients
beta_df = paired_end_experiments_models_denoised.millipede_model_specification_set_with_results['joint_replicate_per_experiment_models'].millipede_model_specification_result_input[0].millipede_model_specification_single_matrix_result[cmm.MillipedeModelType.NORMAL_SIGMA_SCALED].beta
pip_df = paired_end_experiments_models_denoised.millipede_model_specification_set_with_results['joint_replicate_per_experiment_models'].millipede_model_specification_result_input[0].millipede_model_specification_single_matrix_result[cmm.MillipedeModelType.NORMAL_SIGMA_SCALED].pip
display(beta_df.sort_values(ascending=False).head(40))
display(pip_df.sort_values(ascending=False).head(40))

229C>T                 0.300398
154G>A                 0.272165
201G>A                 0.200340
Intercept              0.195948
227C>T                 0.127153
221G>A                 0.077451
202G>A                 0.067098
214G>A                 0.062200
184G>A                 0.048890
140C>T                 0.029899
174C>T                 0.028112
27C>T                  0.022725
181C>T                 0.013418
122C>T                 0.012518
238G>A                 0.011478
134G>A                 0.011130
200G>A                 0.010669
139C>T                 0.008606
intercept_exp0_rep2    0.006843
166G>A                 0.006438
188G>A                 0.006293
182C>T                 0.006020
173C>T                 0.004676
149C>T                 0.004529
intercept_exp0_rep0    0.003964
203G>A                 0.003687
231G>A                 0.003679
123C>T                 0.003619
156C>T                 0.002889
172G>A                 0.002697
135C>T                 0.002272
175C>T  

154G>A    0.999990
229C>T    0.811464
201G>A    0.582983
227C>T    0.417559
214G>A    0.265081
221G>A    0.251152
174C>T    0.142867
27C>T     0.142571
202G>A    0.128172
184G>A    0.078540
140C>T    0.058378
238G>A    0.046976
271G>A    0.041889
29G>A     0.041766
24G>A     0.041099
134G>A    0.040080
139C>T    0.030038
122C>T    0.029649
181C>T    0.028574
200G>A    0.027193
109C>T    0.024795
149C>T    0.024633
182C>T    0.022771
108C>T    0.022683
208C>T    0.022292
111C>T    0.022267
173C>T    0.021168
166G>A    0.020948
188G>A    0.019319
259C>T    0.018685
231G>A    0.017941
263G>A    0.017280
269G>A    0.016883
34G>A     0.015577
197C>T    0.015268
132C>T    0.014759
222G>A    0.014657
172G>A    0.014538
210C>T    0.014368
123C>T    0.014300
Name: PIP, dtype: float64

In [120]:
sigma_hit_table = paired_end_experiments_models_denoised.millipede_model_specification_set_with_results["joint_replicate_per_experiment_models"].millipede_model_specification_result_input[0].millipede_model_specification_single_matrix_result[cmm.MillipedeModelType.NORMAL_SIGMA_SCALED].summary
sigma_hit_table.to_csv('CD19_evoCDA_sigma_hit_table_ZP.csv', index=True)
sigma_hit_table

,PIP,Coefficient,Coefficient StdDev,Conditional Coefficient,Conditional Coefficient StdDev
24G>A,0.028416,-0.011663,0.069974,-0.328111,0.184160
25G>A,0.006584,-0.000977,0.020014,-0.149845,0.197826
26G>A,0.005223,-0.000565,0.024931,-0.053036,0.235704
27C>T,0.010591,0.002447,0.022295,0.171936,0.076095
29G>A,0.031958,-0.018400,0.102294,-0.459131,0.242402
...,...,...,...,...,...
272G>A,0.005834,0.000517,0.022364,0.060438,0.234307
intercept_exp0_rep0,NaN,0.002901,0.027327,0.002901,0.027327
intercept_exp0_rep1,NaN,-0.007188,0.027394,-0.007188,0.027394
intercept_exp0_rep2,NaN,0.005613,0.028612,0.005613,0.028612
